In [ ]:
script "g3/setup.pl";

In [ ]:
print $genus;

In [ ]:
print rows_labeled($points);

In [ ]:
print $triangulations_file;

In [ ]:
@T=();
open (TOPCOM, "<:encoding(UTF-8)", "$triangulations_file");
foreach (<TOPCOM>) { chomp; push @T, new Array<Set>(eval($_)); }

In [ ]:
print $T[0];

In [ ]:
$subdiv = new fan::SubdivisionOfPoints(POINTS=>$points,MAXIMAL_CELLS=>$T[0]);
$sec = $subdiv->secondary_cone;        

In [ ]:
print $sec->RAYS;

In [ ]:
$sec0 = load("g3/secondary_fan/g3b0/g3b0-0.cone");
print $sec0->$_, " " for "DIM", "AMBIENT_DIM";

In [ ]:
print $sec0->type->full_name;

In [ ]:
print $sec0->get_attachment("INDUCED_TRIANGULATION");

In [ ]:
$mod0 = load("g3/secondary_fan/g3b0/moduli/g3b0-0-moduli.cone");
print $mod0->$_, " " for "DIM", "AMBIENT_DIM";

In [ ]:
print join " ", $mod0->list_attachments();

In [ ]:
print $mod0->get_attachment("ID");

In [ ]:
print $mod0->RAYS;

In [ ]:
print rows_numbered($mod0->INPUT_RAYS);

In [ ]:
print $mod0->get_attachment("ZERO_ROW_IDX");

In [ ]:
print rows_numbered($sec0->get_attachment("STANDARD_RAYS"));